In [6]:
import zipfile as zp
import pandas as pd
# from pypac import PACSession as Session #or use requests below if non-ONS
#from requests import Session
from io import BytesIO
import os
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import LoadData as dataLoader
%matplotlib notebook

###  Read in files


In [7]:
# Path to zip files
path = "/Users/lb8075/Loneliness_Data_challenge/Loneliness_notebook/Data/"

In [8]:
print (path[:-5])
print (path[:-25])

/Users/lb8075/Loneliness_Data_challenge/Loneliness_notebook/
/Users/lb8075/Loneliness_Data_challenge/


First lets load all the data... do this just once!

In [9]:
proc_data, proc_data_dict =dataLoader.loadFinalData('../Loneliness')
codes=dataLoader.loadProccessedData('../Loneliness')
drugs_list = dataLoader.loadDrugsList('../Loneliness')
msoa_data, msoa_data_dict = dataLoader.loadmsoaData('../Loneliness')

KeyboardInterrupt: 

In [ ]:
merged_data= proc_data.merge( codes , on=['PCT','pcstrip','SHA'])

In [ ]:
merged_data.head()

In [ ]:
proc_data.rename(index=str, columns={"pcstrip": "Postcode"})

In [ ]:
proc_data = proc_data.merge(codes, on='Postcode') 

In [ ]:
# proc_data=proc_data.loc[:,['oseast1m','osnrth1m','msoa11','insomnia_zscore','alzheimers_zscore','addiction_zscore','depression_zscore']]
# proc_data=proc_data.iloc[1:10000]

In [ ]:
insom=proc_data['insomnia_zscore']

In [ ]:
# proc_data['insomnia_zscore'].hist(bins=100,range=(-2,15))
# print (proc_data['insomnia_zscore'].mean())

In [ ]:
proc_data.dtypes

In [ ]:
geo_data = gpd.GeoDataFrame(proc_data, 
                            crs = {'init':'epsg:27700'}, 
                            geometry = proc_data.apply(lambda geom: Point(geom['oseast1m'],geom['osnrth1m']),axis=1))
geo_data.head(100)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize = (16,6), sharex = True, sharey = True)

geo_data[geo_data['Year'] == 2016].plot(column = 'loneills', scheme = 'quantiles', cmap = 'Reds', marker = '.', ax = ax1);
geo_data[geo_data['Year'] == 2017].plot(column = 'loneills', scheme = 'quantiles', cmap = 'Reds', marker = '.', ax = ax2);
geo_data[geo_data['Year'] == 2018].plot(column = 'loneills', scheme = 'quantiles', cmap = 'Reds', marker = '.', ax = ax3);
